In [16]:
import os
import platform
import torch
import gc
from pprint import pprint

dtype = torch.float64
torch.set_default_dtype(dtype) 

def clear():
    """清空終端畫面"""
    if platform.system() == "Windows":
        os.system("cls")
    else:
        os.system("clear")
cls = clear

def clear_all():
    """
    清除全域變數中常用的資料型態變數 (int, float, str, list, dict, torch.Tensor)
    """
    for name, val in list(globals().items()):
        if isinstance(val, (int, float, str, list, dict, torch.Tensor)):
            del globals()[name]

def cleanup_memory():
    """
    強制垃圾回收並清空 PyTorch CUDA 記憶體
    """
    gc.collect()
    torch.cuda.empty_cache()
cm = cleanup_memory

def convert_all_tensors_dtype(dtype=dtype):
    for obj in gc.get_objects():
        if isinstance(obj, torch.Tensor):
            if obj.dtype != dtype:
                obj.data = obj.data.to(dtype=dtype)

def p(obj):
    """
    遞迴漂亮印出 dict / list / tensor。
    tensor 會換行縮排對齊，方便查看。
    """
    def pretty_tensor(tensor: torch.Tensor, indent: int = 6) -> str:
        lines = str(tensor).split("\n")
        if len(lines) == 1:
            return f"{lines[0]}"
        ind = " " * indent
        return "\n" + "\n".join(ind + line for line in lines) + "\n" + " " * (indent - 2)

    def _p(obj, indent=0):
        space = " " * indent
        if isinstance(obj, dict):
            print(space + "{")
            for k, v in obj.items():
                print(f"{space}  {repr(k)}: ", end="")
                _p(v, indent + 4)
            print(space + "}")
        elif isinstance(obj, list) or isinstance(obj, tuple):
            print(space + "[")
            for v in obj:
                _p(v, indent + 4)
            print(space + "]")
        elif isinstance(obj, torch.Tensor):
            print(pretty_tensor(obj, indent + 4))
        else:
            print(repr(obj))

    _p(obj)


In [17]:
convert_all_tensors_dtype(dtype=dtype)

C:\Users\user\AppData\Local\Temp\ipykernel_22724\2859990534.py:36: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  if isinstance(obj, torch.Tensor):


In [18]:
import logging
import colorlog

# logger config
def setup_logger() -> logging.Logger:
    logger = logging.getLogger(__name__)

    # Check if logger has already been configured
    if logger.hasHandlers():
        return logger

    logger.setLevel(logging.INFO)

    # Create a formatter
    formatter = colorlog.ColoredFormatter(
        fmt='%(log_color)s%(asctime)s - %(name)s - %(levelname)s: %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        log_colors={
            'DEBUG': 'cyan',
            'INFO': 'green',
            'WARNING': 'yellow',
            'ERROR': 'red',
            'CRITICAL': 'bold_red',
        }
    )

    # Create a console handler
    console_handler = colorlog.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(formatter)

    # Add the handler to the logger
    logger.addHandler(console_handler)

    return logger


In [19]:
"""
Title: Setup file of autoFRK-Python Project
Author: Hsu, Yao-Chih
Version: 1141007
Reference:
"""

# development only
import os
import sys
sys.path.append(os.path.abspath("./src"))

# import modules
import os
import torch
import numpy as np
import faiss
import gc
from typing import Optional, Union
from sklearn.neighbors import NearestNeighbors

# logger config
LOGGER = setup_logger()

# fast mode KNN for missing data imputation, using in autoFRK
# Its have OpenMP issue, set environment variable OMP_NUM_THREADS=1 to avoid it, or use sklearn version below
# check = ok
def fast_mode_knn(
    data: torch.Tensor,
    loc: torch.Tensor, 
    n_neighbor: int = 3
) -> torch.Tensor:
    """
    The fast mode for autoFRK by using KNN for missing data imputation.

    Parameters:
        data: (N, T) or (samples, time_points) tensor.
        loc: (N, spatial_dim) tensor, e.g., 2D space N x 2.
        n_neighbor: Number of neighbors to use for KNN.
        
    Returns:
        torch.Tensor: The data tensor with missing values imputed.
    """
    dtype=data.dtype
    device=data.device

    data = data.detach().cpu().numpy()
    loc = loc.detach().cpu().numpy().astype(np.float32)

    # use faiss on GPU if available
    if device.type != 'cpu':
        res = faiss.StandardGpuResources()

    for tt in range(data.shape[1]):
        col = data[:, tt]
        where = np.isnan(col)
        if not np.any(where):
            continue

        known_idx = np.where(~where)[0]
        unknown_idx = np.where(where)[0]

        # if low known values
        if len(known_idx) < n_neighbor:
            err_msg = f'Column {tt} has too few known values to impute ({len(known_idx)} < {n_neighbor}).'
            LOGGER.warning(err_msg)
            raise ValueError(err_msg)

        # use faiss for KNN
        index = faiss.IndexFlatL2(loc.shape[1])
        if device.type != 'cpu':
            index = faiss.index_cpu_to_gpu(res, 0, index)

        # get the values of neighbors
        index.add(loc[known_idx])
        _, knn_idx = index.search(loc[unknown_idx], n_neighbor)

        # impute missing values with the mean of neighbors
        neighbor_vals = col[known_idx[knn_idx]]
        col[where] = np.nanmean(neighbor_vals, axis=1)
        data[:, tt] = col

    return torch.tensor(data, dtype=dtype, device=device)

# fast mode KNN for missing data imputation, using in autoFRK, sklearn version
# check = ok
def fast_mode_knn_sklearn(
    data: torch.Tensor,
    loc: torch.Tensor,
    n_neighbor: int = 3
) -> torch.Tensor:
    
    dtype = data.dtype
    device = data.device

    data = data.detach().cpu().numpy()
    loc = loc.detach().cpu().numpy()

    for tt in range(data.shape[1]):
        col = data[:, tt]
        where = np.isnan(col)
        if not np.any(where):
            continue

        known_idx = np.where(~where)[0]
        unknown_idx = np.where(where)[0]

        if 0 < len(known_idx) < n_neighbor:
            err_msg = f'Column {tt} has too few known values to impute ({len(known_idx)} < {n_neighbor}).'
            LOGGER.warning(err_msg)
            raise ValueError(err_msg)

        knn = NearestNeighbors(n_neighbors=n_neighbor, algorithm='auto').fit(loc[known_idx])
        distances, knn_idx = knn.kneighbors(loc[unknown_idx])

        neighbor_vals = col[known_idx[knn_idx]]
        col[where] = np.nanmean(neighbor_vals, axis=1)
        data[:, tt] = col

    return torch.tensor(data, dtype=dtype, device=device)

# select basis function for autoFRK, using in autoFRK
# check = none
def selectBasis(
    data: torch.Tensor,
    loc: torch.Tensor,
    D: torch.Tensor = None,
    maxit: int = 50,
    avgtol: float = 1e-6,
    max_rank: int = None,
    sequence_rank: torch.Tensor = None,
    method: str = "fast",
    num_neighbors: int = 3,
    max_knot: int = 5000,
    DfromLK: dict = None,
    Fk: torch.Tensor = None,
    device: Optional[Union[torch.device, str]] = 'cpu'
) -> torch.Tensor:
    # 去除全為 NaN 的欄位
    not_all_nan = ~torch.isnan(data).all(dim=0)
    data = data[:, not_all_nan]

    # 檢查資料中是否有缺失值
    is_data_with_missing_values = torch.isnan(data).any()

    # 找出整行都是 NaN 的列（完全缺失）
    na_rows = torch.isnan(data).all(dim=1)
    pick = torch.arange(data.shape[0])
    if na_rows.any():
        data = data[~na_rows]
        loc = loc[~na_rows]  # 同步刪除 loc 中相同的行 need fix
        D = D[~na_rows][:, ~na_rows]
        pick = pick[~na_rows]
        is_data_with_missing_values = torch.isnan(data).any()

    # 如果 D 未提供，則初始化為單位對角矩陣
    if D is None:
        D = torch.eye(data.shape[0], device=data.device)

    # 取得位置維度
    d = loc.shape[1]

    # 計算 klim 與選 knot
    N = len(pick)
    klim = int(min(N, np.round(10 * np.sqrt(N))))
    if N < max_knot:
        knot = loc[pick, :]
    else:
        knot = subKnot(x=loc[pick, :],
                       nknot=min(max_knot, klim),
                       device=device
                       ).to(device=device)

    # 處理 K 值
    if max_rank is not None:
        max_rank = round(max_rank)
    else:
        max_rank = torch.round(torch.max(sequence_rank)).to(torch.int) if sequence_rank is not None else klim

    if sequence_rank is not None:
        K = torch.unique(torch.round(sequence_rank).to(torch.int))
        if K.max() > max_rank:
            err_msg = f'maximum of sequence_rank is larger than max_rank!'
            LOGGER.error(err_msg)
            raise ValueError(err_msg)
        elif torch.all(K <= d):
            err_msg = f'Not valid sequence_rank!'
            LOGGER.error(err_msg)
            raise ValueError(err_msg)
        elif torch.any(K < (d + 1)):
            warn_msg = f'The minimum of sequence_rank can not less than {d + 1}. Too small values will be ignored.'
            LOGGER.warning(warn_msg)
        K = K[K > d]
    else:
        step = max_rank ** (1/3) * d
        K = torch.arange(d + 1, max_rank, step).round().to(torch.int).unique()
        if len(K) > 30:
            K = torch.linspace(d + 1, max_rank, 30).round().to(torch.int).unique()

    # Fk 為 None 時初始化 basis function 值
    if Fk is None:
        mrts = MRTS(locs=loc, k=max(K), device=device)  # 待修 (knot, max(K), loc, max_knot) need fix
        Fk = mrts.forward()

    AIC_list = [float('inf')] * len(K)
    num_data_columns = data.shape[1]

    if method == "EM" and DfromLK is None:
        for k in range(len(K)):
            AIC_list[k] = indeMLE(data,
                                  Fk[pick, :K[k]],
                                  D,
                                  maxit,
                                  avgtol,
                                  wSave=False,
                                  verbose=False
                                  )["negloglik"]
    else:
        if is_data_with_missing_values:
            data = fast_mode_knn_sklearn(data=data, loc=loc, n_neighbor=num_neighbors) 
        if DfromLK is None:
            iD = torch.linalg.solve(D, torch.eye(D.shape[0], device=D.device))
            iDFk = iD @ Fk[pick, :]
            iDZ = iD @ data
        else:
            wX = DfromLK["wX"][pick, :]
            G = DfromLK["wX"].T @ DfromLK["wX"] + DfromLK["lambda"] * DfromLK["Q"]
            weight = DfromLK["weights"][pick]
            wwX = torch.diag(torch.sqrt(weight)) @ wX
            wXiG = torch.linalg.solve(G, wwX.T).T
            iDFk = weight * Fk[pick, :] - wXiG @ (wwX.T @ Fk[pick, :])
            iDZ = weight * data - wXiG @ (wwX.T @ data)

        sample_covariance_trace = torch.sum(iDZ * data) / num_data_columns

        for k in range(len(K)):
            Fk_k = Fk[pick, :K[k]]
            iDFk_k = iDFk[:, :K[k]]
            inverse_square_root_matrix = get_inverse_square_root_matrix(Fk_k, iDFk_k)
            ihFiD = inverse_square_root_matrix @ iDFk_k.T
            tmp = torch.matmul(ihFiD, data)
            matrix_JSJ = torch.matmul(tmp, tmp.T) / num_data_columns
            matrix_JSJ = (matrix_JSJ + matrix_JSJ.T) / 2
            AIC_list[k] = cMLE(Fk=Fk_k,
                               num_columns=num_data_columns,
                               sample_covariance_trace=sample_covariance_trace,
                               inverse_square_root_matrix=inverse_square_root_matrix,
                               matrix_JSJ=matrix_JSJ
                               )["negloglik"]

    # 計算 AIC 並選出最佳 K 值
    df = torch.where(
        K <= num_data_columns,
        (K * (K + 1) / 2 + 1),
        (K * num_data_columns + 1 - num_data_columns * (num_data_columns - 1) / 2)
    )

    AIC_list = AIC_list + 2 * df
    Kopt = K[torch.argmin(AIC_list)].item()
    out = Fk[:, :Kopt]
    return out

# check = ok
def get_inverse_square_root_matrix(left_matrix, right_matrix):
    mat = left_matrix.T @ right_matrix  # A^T * B
    mat = (mat + mat.T) / 2
    eigvals, eigvecs = torch.linalg.eigh(mat)
    inv_sqrt_eigvals = torch.diag(torch.clamp(eigvals, min=1e-10).rsqrt())
    return eigvecs @ inv_sqrt_eigvals @ eigvecs.T

# subset knot selection for autoFRK, using in selectBasis
# check = ok
def subKnot(
    x: torch.Tensor, 
    nknot: int, 
    xrng: torch.Tensor = None, 
    nsamp: int = 1, 
    device: Optional[Union[torch.device, str]]='cpu'
) -> torch.Tensor:
    x = x.to(device)
    x = torch.sort(x, dim=0).values
    xdim = x.shape  # (N, D)

    if xrng is None:
        xrng = torch.stack([x.min(dim=0).values, x.max(dim=0).values], dim=0)

    rng = torch.sqrt(xrng[1] - xrng[0])
    if (rng == 0).any():
        rng[rng == 0] = rng[rng > 0].min() / 5
    rng = rng * 10 / rng.min()
    rng_max_index = torch.argmax(rng).item()

    log_rng = torch.log(rng)
    nmbin = torch.round(torch.exp(log_rng * torch.log(torch.tensor(nknot, dtype=torch.float32)) / log_rng.sum())).int()
    nmbin = torch.clamp(nmbin, min=2)

    while torch.prod(nmbin).item() < nknot:
        nmbin[rng_max_index] += 1

    gvec = torch.ones(xdim[0], dtype=torch.int64, device=device)
    cnt = 0
    while len(torch.unique(gvec)) < nknot:
        nmbin += cnt
        kconst = 1
        gvec = torch.ones(xdim[0], dtype=torch.int64, device=device)
        for kk in range(xdim[1]):
            delta = xrng[1, kk] - xrng[0, kk]
            if delta == 0:
                grp = torch.zeros(xdim[0], dtype=torch.int64, device=device)
            else:
                grp = ((nmbin[kk] - 1) * (x[:, kk] - xrng[0, kk]) / delta).round().int()
                grp = torch.clamp(grp, max=nmbin[kk] - 1)

            if len(torch.unique(grp)) < nmbin[kk]:
                brk = torch.tensor(np.quantile(x[:, kk].cpu().numpy(), np.linspace(0, 1, nmbin[kk] + 1)), device=device)
                brk[0] -= 1e-8
                grp = torch.bucketize(x[:, kk], brk) - 1
            gvec += kconst * grp
            kconst *= nmbin[kk]

        cnt += 1

    gvec_np = gvec.cpu().numpy()
    index = []
    for g in np.unique(gvec_np):
        idx = np.where(gvec_np == g)[0]
        if len(idx) == 1:
            index.append(idx[0])
        else:
            np.random.seed(int(np.mean(idx)))
            index.extend(np.random.choice(idx, size=min(nsamp, len(idx)), replace=False))

    index = torch.tensor(index, device=device)
    return x[index]

# compute negative log likelihood for autoFRK, using in selectBasis
# check = none
def cMLE(
    Fk: torch.Tensor,
    num_columns: int,
    sample_covariance_trace: float,
    inverse_square_root_matrix: torch.Tensor,
    matrix_JSJ: torch.Tensor,
    s: float = 0,
    ldet: float = 0,
    wSave: bool = False,
    onlylogLike: bool = None,
    vfixed: float = None,
    device: Optional[Union[torch.device, str]] = 'cpu'
) -> dict:
    """
    Internal function: maximum likelihood estimate with the likelihood

    Parameters:
        Fk: (N, K) torch.Tensor, basis functions.
        num_columns: (int) Number of columns in the data.
        sample_covariance_trace: (float) Trace of the sample covariance matrix.
        inverse_square_root_matrix: (N, K) torch.Tensor, inverse square root matrix.
        matrix_JSJ: (K, K) torch.Tensor, covariance-like matrix.
        s: (float) Effective sample size, default is 0.
        ldet: (float) Log determinant of the transformation matrix, default is 0.
        wSave: (bool) Whether to save the L matrix, default is False.
        onlylogLike: (bool) If True, only return the negative log likelihood.
        vfixed: (float, optional) Fixed noise variance, if provided.
        device: (str) 'cpu' or 'cuda'.

    Returns:
        dict: {
            'v': (float) Estimated noise variance,
            'M': (torch.Tensor) Matrix M,
            's': (int) Effective sample size,
            'negloglik': (float) Negative log likelihood,
            'L': (torch.Tensor, optional) L matrix if wSave is True.
        }
    """
    nrow_Fk = Fk.shape[0]

    likelihood_object = computeNegativeLikelihood(
        nrow_Fk=nrow_Fk,
        ncol_Fk=Fk.shape[1],
        s=s,
        p=num_columns,
        matrix_JSJ=matrix_JSJ,
        sample_covariance_trace=sample_covariance_trace,
        vfixed=vfixed,
        ldet=ldet,
        device=device
    )

    negative_log_likelihood = likelihood_object['negative_log_likelihood']

    if onlylogLike:
        return {'negloglik': negative_log_likelihood}

    P = likelihood_object['P']
    d_hat = likelihood_object['d_hat']
    v = likelihood_object['v']
    M = inverse_square_root_matrix @ P @ (d_hat * P.T) @ inverse_square_root_matrix

    if not wSave:
        L = None
    elif d_hat[0] != 0:
        L = Fk @ ((torch.sqrt(d_hat) * P.T) @ inverse_square_root_matrix)
        L = L[:, d_hat > 0]
    else:
        L = torch.zeros((nrow_Fk, 1), dtype=Fk.dtype, device=Fk.device)

    return {'v': v,
            'M': M,
            's': s,
            'negloglik': negative_log_likelihood,
            'L': L
            }

# compute negative log likelihood for autoFRK, using in cMLE
# check = ok
def computeNegativeLikelihood(
    nrow_Fk: int,
    ncol_Fk: int,
    s: int,
    p: int,
    matrix_JSJ: torch.Tensor,
    sample_covariance_trace: float,
    vfixed: float = None,
    ldet: float = 0.0,
    device: Optional[Union[torch.device, str]]='cpu'
) -> dict:
    """
    Compute negative log-likelihood.
    
    Parameters:
        nrow_Fk: (int) Number of rows in basis matrix Fk.
        ncol_Fk: (int) Number of basis functions (columns of Fk).
        s: (int) Effective sample size.
        p: (int) Number of variables (e.g. number of spatial points).
        matrix_JSJ: (torch.Tensor) Covariance-like matrix (should be symmetric).
        sample_covariance_trace: (float) Trace of sample covariance matrix.
        vfixed: (float, optional) Fixed noise variance (if provided).
        ldet: (float, optional) Log determinant of transformation matrix.
        device: (str) 'cpu' or 'cuda'.

    Returns:
        dict: {
            'negative_log_likelihood': float,
            'P': torch.Tensor (eigenvectors),
            'v': float,
            'd_hat': torch.Tensor
        }
    """
    matrix_JSJ = matrix_JSJ.to(device)

    if not torch.allclose(matrix_JSJ, matrix_JSJ.T, atol=1e-10):
        err_msg = f'Please input a symmetric matrix'
        LOGGER.error(err_msg)
        raise ValueError(err_msg)

    if matrix_JSJ.size(1) < ncol_Fk:
        err_msg = f'Please input the rank of a matrix larger than ncol_Fk = {ncol_Fk}'
        LOGGER.error(err_msg)
        raise ValueError(err_msg)

    eigenvalues_JSJ, eigenvectors_JSJ = torch.linalg.eigh(matrix_JSJ)
    idx = torch.argsort(eigenvalues_JSJ, descending=True)
    eigenvalues_JSJ = eigenvalues_JSJ[idx][:ncol_Fk]
    eigenvectors_JSJ = eigenvectors_JSJ[:, idx][:, :ncol_Fk]

    if vfixed is None:
        v = estimateV(d=eigenvalues_JSJ, 
                      s=s, 
                      sample_covariance_trace=sample_covariance_trace, 
                      n=nrow_Fk
                      )
    else:
        v = vfixed

    d = torch.clamp(eigenvalues_JSJ, min=0)
    d_hat = estimateEta(d, s, v)

    negative_log_likelihood = neg2llik(d=d, 
                                       s=s, 
                                       v=v, 
                                       sample_covariance_trace=sample_covariance_trace, 
                                       sample_size=nrow_Fk
                                       ) * p + ldet * p

    return {"negative_log_likelihood": negative_log_likelihood,
            "P": eigenvectors_JSJ,
            "v": v,
            "d_hat": d_hat
            }

# estimate the eta parameter for negative likelihood, using in computeNegativeLikelihood
# check = ok
def estimateV(
    d: torch.Tensor, 
    s: float, 
    sample_covariance_trace: float, 
    n: int
) -> float:
    """
    Estimate the v parameter.

    Parameters:
        d: (torch.Tensor) 1D tensor of nonnegative eigenvalues (length k)
        s: (float) A positive numeric constant
        sample_covariance_trace: (float) Trace of sample covariance
        n: (int) Sample size

    Returns:
        v: (float) Estimated noise variance
    """
    if torch.max(d) < max(sample_covariance_trace / n, s):
        return max(sample_covariance_trace / n - s, 0.0)

    k = d.shape[0]
    cumulative_d_values = torch.cumsum(d, dim=0)
    ks = torch.arange(1, k + 1, device=d.device)
    if k == n:
        ks[-1] = n - 1

    eligible_indices = torch.nonzero(d > (sample_covariance_trace - cumulative_d_values) / (n - ks)).flatten()
    
    if len(eligible_indices) == 0:
        error_msg = "No eligible indices found: check input d, sample_covariance_trace, and n."
        LOGGER.error(error_msg)
        raise ValueError(error_msg)
    L = int(torch.max(eligible_indices))

    if (L + 1) >= n:
        L = n - 1
        v_hat = max((sample_covariance_trace - cumulative_d_values[L - 1]) / (n - L) - s, 0.0)
    else:
        v_hat = max((sample_covariance_trace - cumulative_d_values[L]) / (n - L - 1) - s, 0.0)
    return v_hat

# estimate the eta parameter for negative likelihood, using in computeNegativeLikelihood
# check = ok
def estimateEta(
    d: torch.Tensor, 
    s: float, 
    v: float
) -> torch.Tensor:
    """
    Estimate the eta parameter.

    Parameters:
        d: (torch.Tensor) 1D tensor of nonnegative values (eigenvalues)
        s: (float) A positive numeric
        v: (float) A positive numeric

    Returns:
        torch.Tensor: A tensor of estimated eta values
    """
    return torch.clamp(d - s - v, min=0.0)

# compute the negative log likelihood, using in computeNegativeLikelihood
# check = ok
def neg2llik(
    d: torch.Tensor,
    s: float,
    v: float,
    sample_covariance_trace: float,
    sample_size: int
) -> float:
    """
    Estimate the negative log-likelihood (up to constant)

    Parameters:
        d: Tensor of nonnegative values (eigenvalues)
        s: A positive scalar
        v: A positive scalar
        sample_covariance_trace: Scalar trace value
        sample_size: Number of samples (int)

    Returns:
        Scalar negative log-likelihood value
    """
    k = d.shape[0]
    eta = estimateEta(d, s, v)

    if torch.max(eta / (s + v)) > 1e20:
        return float("inf")
    sPlusv = torch.as_tensor(s + v, device=d.device, dtype=d.dtype)
    log_det_term = torch.sum(torch.log(eta + sPlusv))
    log_sv_term = torch.log(sPlusv) * (sample_size - k)
    trace_term = sample_covariance_trace / (sPlusv)
    eta_term = torch.sum(d * eta / (eta + sPlusv)) / (sPlusv)

    return sample_size * torch.log(torch.tensor(2 * torch.pi, device=d.device, dtype=d.dtype)) + log_det_term + log_sv_term + trace_term - eta_term

# independent maximum likelihood estimation for autoFRK, using in selectBasis
# check = none
def indeMLE(
    data: torch.Tensor,
    Fk: torch.Tensor,
    D: Optional[torch.Tensor] = None,
    maxit: int = 50,
    avgtol: float = 1e-6,
    wSave: bool = False,
    DfromLK: Optional[dict] = None,
    vfixed: Optional[float] = None,
    verbose: bool = True,
    device: Optional[Union[torch.device, str]]='cpu'
) -> dict:
    """
    
    """
    device = torch.device(device)
    data = data.to(device)
    Fk = Fk.to(device)

    withNA = torch.isnan(data).any().item()

    TT = data.shape[1]
    empty = torch.isnan(data).all(dim=0)
    notempty = (~empty).nonzero(as_tuple=True)[0]
    if empty.any():
        data = data[:, notempty]

    del_rows = torch.isnan(data).all(dim=1).nonzero(as_tuple=True)[0]
    pick = torch.arange(data.shape[0], device=device)

    if D is None:
        D = torch.eye(data.shape[0], device=device).to_sparse()

    if not torch.allclose(D, torch.diag(torch.diagonal(D))):
        D0 = D
    else:
        D0 = torch.diag(torch.diag(D)).to_sparse()

    if withNA and len(del_rows) > 0:
        pick = pick[~torch.isin(pick, del_rows)]
        data = data[~torch.isin(torch.arange(data.shape[0], device=device), del_rows), :]
        Fk = Fk[~torch.isin(torch.arange(Fk.shape[0], device=device), del_rows), :]
        if not torch.allclose(D, torch.diag(torch.diagonal(D))):
            D = D[~torch.isin(torch.arange(D.shape[0], device=device), del_rows)][:, ~torch.isin(torch.arange(D.shape[1], device=device), del_rows)]
        else:
            keep_mask = ~torch.isin(torch.arange(D.shape[0], device=device), del_rows)
            full_diag = torch.zeros(D.shape[0], device=device)
            full_diag[keep_mask] = torch.diagonal(D)[keep_mask]
            D = torch.diag(full_diag)
        withNA = torch.isnan(data).any().item()

    N = data.shape[0]
    K = Fk.shape[1]
    Depsilon = D
    is_diag = torch.allclose(D, torch.diag(torch.diagonal(D)))
    mean_diag = torch.mean(torch.diagonal(D))
    isimat = is_diag and torch.allclose(torch.diagonal(Depsilon), mean_diag.repeat(N), atol=1e-10)

    if not withNA:
        if isimat and DfromLK is None:
            sigma = 0  # we cannot find `.Option$sigma_FRK` in the R code # need fix
            out = cMLEimat(Fk, 
                           data, 
                           s=sigma, 
                           wSave=wSave
                           )
            if out['v'] is not None:
                out['s'] = out['v'] if sigma == 0 else sigma
                del out['v']
            if wSave:
                w = torch.zeros((K, TT), device=device)
                w[:, notempty] = out['w']
                out['w'] = w
                out['pinfo'] = {'D': D0, 
                                'pick': pick
                                }
            return out
        elif DfromLK is None:
            out = cMLEsp(Fk, 
                         data, 
                         Depsilon, 
                         wSave
                         )
            if wSave:
                w = torch.zeros((K, TT), device=device)
                w[:, notempty] = out['w']
                out['w'] = w
                out['pinfo'] = {'D': D0, 
                                'pick': pick
                                }
            return out
        else:
            out = cMLElk(Fk, 
                         data, 
                         Depsilon, 
                         wSave, 
                         DfromLK, 
                         vfixed
                         )
            if wSave:
                w = torch.zeros((K, TT), device=device)
                w[:, notempty] = out['w']
                out['w'] = w
            return out
    else:
        out = EM0miss(Fk, 
                      data, 
                      Depsilon, 
                      maxit, 
                      avgtol, 
                      wSave,
                      DfromLK=DfromLK, 
                      vfixed=vfixed, 
                      verbose=verbose
                      )
        if wSave:
            w = torch.zeros((K, TT), device=device)
            w[:, notempty] = out['w']
            out['w'] = w
            if DfromLK is None:
                out['pinfo'] = {'D': D0,
                                'pick': pick
                                }
        return out

# convert dense tensor to sparse matrix, using in indeMLE
# python 不需要，在 R 中僅作為節省記憶體的角色
# def toSparseMatrix(
#     mat: torch.Tensor, 
#     verbose: bool=False
# ) -> torch.Tensor:
#     """
    
#     """
#     if not torch.is_tensor(mat):
#         warn_msg = f'Expected tensor, but got {type(mat)}'
#         LOGGER.warning(warn_msg)
#         mat = torch.tensor(mat)
    
#     if mat.is_sparse:
#         if verbose:
#             info_msg = f'The input is already a sparse tensor'
#             LOGGER.info(info_msg)
#         return mat

#     if verbose:
#         return mat.to_sparse()

# using in indeMLE
# check = ok
def cMLEimat(
    Fk: torch.Tensor,
    data: torch.Tensor,
    s: float,
    wSave: bool = False,
    S: Optional[torch.Tensor] = None,
    onlylogLike: Optional[bool] = None,
    device: Optional[Union[torch.device, str]]='cpu'
) -> dict:

    if onlylogLike is None:
        onlylogLike = not wSave

    data = data.to(device)
    Fk = Fk.to(device)

    num_columns = data.shape[1]
    nrow_Fk, ncol_Fk = Fk.shape

    projection = computeProjectionMatrix(Fk1=Fk, 
                                         Fk2=Fk, 
                                         data=data, 
                                         S=S, 
                                         device=device
                                         )
    inverse_square_root_matrix = projection["inverse_square_root_matrix"]
    matrix_JSJ = projection["matrix_JSJ"]

    sample_covariance_trace = torch.sum(data ** 2) / num_columns

    likelihood_object = computeNegativeLikelihood(nrow_Fk=nrow_Fk,
                                                  ncol_Fk=ncol_Fk,
                                                  s=s,
                                                  p=num_columns,
                                                  matrix_JSJ=matrix_JSJ,
                                                  sample_covariance_trace=sample_covariance_trace,
                                                  device=device
                                                  )

    negative_log_likelihood = likelihood_object["negative_log_likelihood"]

    if onlylogLike:
        return {"negloglik": negative_log_likelihood}

    P = likelihood_object["P"]
    d_hat = likelihood_object["d_hat"]
    v = likelihood_object["v"]

    M = inverse_square_root_matrix @ P @ (P.T * d_hat[:, None]) @ inverse_square_root_matrix

    if not wSave:
        return {"v": v, 
                "M": M, 
                "s": s, 
                "negloglik": negative_log_likelihood
                }

    L = Fk @ ((torch.diag(torch.sqrt(d_hat)) @ P.T) @ inverse_square_root_matrix).T

    if ncol_Fk > 2:
        reduced_columns = torch.cat([
            torch.tensor([0], device=device),
            (d_hat[1:(ncol_Fk - 1)] > 0).nonzero(as_tuple=True)[0]
        ])
    else:
        reduced_columns = torch.tensor([ncol_Fk - 1], device=device)

    L = L[:, reduced_columns]

    invD = torch.ones(nrow_Fk, device=device) / (s + v)
    iDZ = invD[:, None] * data

    right = L @ (torch.linalg.inv(torch.eye(L.shape[1], device=device) + L.T @ (invD[:, None] * L)) @ (L.T @ iDZ))

    INVtZ = iDZ - invD[:, None] * right
    etatt = M @ Fk.T @ INVtZ

    GM = Fk @ M

    diag_matrix = (s + v) * torch.eye(nrow_Fk, device=device)

    V = M - GM.T @ invCz(R=diag_matrix,
                         L=L, 
                         z=GM,
                         device=device
                         ).T

    return {"v": v,
            "M": M,
            "s": s,
            "negloglik": negative_log_likelihood,
            "w": etatt,
            "V": V
            }

# using in cMLEimat
# check = ok
def computeProjectionMatrix(
    Fk1: torch.Tensor, 
    Fk2: torch.Tensor, 
    data: torch.Tensor, 
    S: torch.Tensor=None, 
    device: Optional[Union[torch.device, str]]='cpu'
) -> dict:
    """
    Internal function: maximum likelihood estimate with the likelihood

    Parameters:
        Fk1 (torch.Tensor): (n, K) matrix
        Fk2 (torch.Tensor): (n, K) matrix
        data (torch.Tensor): (n, T) matrix
        S (torch.Tensor or None): (n, n) matrix
        device (str): "cpu" or "cuda"
 
    Returns:
        dict: {
            'inverse_square_root_matrix': torch.Tensor,
            'matrix_JSJ': torch.Tensor
        }
    """
    Fk1 = Fk1.to(device)
    Fk2 = Fk2.to(device)
    data = data.to(device)
    if S is not None:
        S = S.to(device)

    num_columns = data.shape[1]
    inverse_square_root_matrix = getInverseSquareRootMatrix(A=Fk1, 
                                                            B=Fk2, 
                                                            device=device
                                                            )
    inverse_square_root_on_Fk2 = inverse_square_root_matrix @ Fk2.T

    if S is None:
        matrix_JSJ = (inverse_square_root_on_Fk2 @ data) @ (inverse_square_root_on_Fk2 @ data).T / num_columns
    else:
        matrix_JSJ = (inverse_square_root_on_Fk2 @ S) @ inverse_square_root_on_Fk2.T

    matrix_JSJ = (matrix_JSJ + matrix_JSJ.T) / 2

    return {"inverse_square_root_matrix": inverse_square_root_matrix,
            "matrix_JSJ": matrix_JSJ
            }

# using in computeProjectionMatrix
# check = ok
def getInverseSquareRootMatrix(
    A: torch.Tensor, 
    B: torch.Tensor, 
    device: Optional[Union[torch.device, str]]='cpu',
    eps: float = 1e-10
) -> torch.Tensor:
    """
    Compute inverse square root matrix of (A.T @ B), assuming it is symmetric.
    
    Parameters:
        A: Tensor of shape (n, k)
        B: Tensor of shape (n, k)
        device: 'cpu' or 'cuda'
        
    Returns:
        Inverse square root of (A.T @ B): Tensor of shape (k, k)
    """
    A = A.to(device)
    B = B.to(device)

    mat = A.T @ B

    eigenvalues, eigenvectors = torch.linalg.eigh(mat)
    eigvals_clamped = torch.clamp(eigenvalues, min=eps)
    inv_sqrt_eigvals = torch.diag(eigvals_clamped.rsqrt())

    return eigenvectors @ inv_sqrt_eigvals @ eigenvectors.T

# using in cMLEimat
# check = ok
def invCz(
    R: torch.Tensor, 
    L: torch.Tensor, 
    z: torch.Tensor, 
    device: Optional[Union[torch.device, str]]='cpu'
) -> torch.Tensor:
    """


    Parameters:
        R: (p x p) positive definite matrix
        L: (p x K) matrix
        z: (p,) vector or (1 x p) row matrix
        device: 'cpu' or 'cuda', or torch.device

    Returns:
        (1 x p) tensor
    """

    dtype = R.dtype
    if R.dtype != torch.float64:
        R = R.to(dtype=torch.float64)
        L = L.to(dtype=torch.float64)
        z = z.to(dtype=torch.float64)

    if z.dim() == 1:
        z = z.unsqueeze(1)

    K = L.shape[1]
    iR = torch.linalg.pinv(R)
    iRZ = iR @ z
    right = L @ torch.linalg.inv(torch.eye(K, device=R.device, dtype=torch.float64) + (L.T @ iR @ L)) @ (L.T @ iRZ) 
    result = iRZ - iR @ right

    return result.T.to(dtype=dtype)

# using in indeMLE
# check = ok, but have some problem
def EM0miss(
    Fk: torch.Tensor, 
    data: torch.Tensor, 
    Depsilon: torch.Tensor, 
    maxit: int=100, 
    avgtol: float=1e-4, 
    wSave: bool=False, 
    DfromLK: dict=None,
    vfixed: float=None,
    verbose: bool=True,
    device: Optional[Union[torch.device, str]] = 'cpu'
) -> dict:
    """

    """
    Fk = Fk.to(device)
    data = data.to(device)
    Depsilon = Depsilon.to(device)

    O = ~torch.isnan(data)
    TT = data.shape[1]
    ncol_Fk = Fk.shape[1]

    ziDz = torch.full((TT,), float('nan'), device=device)
    ziDB = torch.full((TT, ncol_Fk), float('nan'), device=device)
    db = {}
    D = Depsilon
    iD = torch.linalg.inv(D)
    diagD = isDiagonal(D)

    if DfromLK is not None:
        pick = DfromLK.get("pick", None)
        weights = DfromLK["weights"].clone().detach()
        if pick is None:
            pick = torch.arange(len(weights), device=device)
        else:
            pick = torch.tensor(pick, dtype=torch.long, device=device)
        weight = weights[pick]

        if not torch.is_tensor(DfromLK["wX"]):
            DfromLK["wX"] = torch.tensor(DfromLK["wX"], device=device)
        DfromLK["wX"] = DfromLK["wX"][pick, :].clone().detach()

        if not torch.is_tensor(DfromLK["Q"]):
            DfromLK["Q"] = torch.tensor(DfromLK["Q"], device=device)

        wwX = torch.diag(torch.sqrt(weight)) @ DfromLK["wX"]
        lQ = DfromLK["lambda"] * DfromLK["Q"].clone().detach()

    for tt in range(TT):
        if DfromLK is not None:
            obs_idx = O[:, tt].bool()
            iDt = None
            if obs_idx.sum() == O.shape[0]:
                wXiG = wwX @ torch.linalg.inv(DfromLK["G"].to(device=device))
            else:
                wX_obs = DfromLK["wX"][obs_idx, :].to(device)
                G = wX_obs.T @ wX_obs + lQ.to(device)
                wXiG = wwX[obs_idx, :] @ torch.linalg.inv(G)

            Bt = Fk[obs_idx, :].to(device)
            if Bt.ndim == 1:
                Bt = Bt.unsqueeze(0)

            iDBt = weight[obs_idx].unsqueeze(1) * Bt - wXiG @ (wwX[obs_idx, :].T @ Bt)
            zt = data[obs_idx, tt].to(device=device)
            ziDz[tt] = torch.sum(zt * (weight[obs_idx] * zt - wXiG @ (wwX[obs_idx, :].T @ zt)))
            ziDB[tt, :] = (zt @ iDBt).squeeze()
            BiDBt = Bt.T @ iDBt

        else:
            if not diagD:
                iDt = torch.linalg.inv(D[obs_idx][:, obs_idx].to(device))
            else:
                iDt = iD[obs_idx][:, obs_idx].to(device)

            Bt = Fk[obs_idx, :].to(device)
            if Bt.ndim == 1:
                Bt = Bt.unsqueeze(0)

            iDBt = iDt @ Bt
            zt = data[obs_idx, tt]
            ziDz[tt] = torch.sum(zt * (iDt @ zt))
            ziDB[tt, :] = (zt @ iDBt).squeeze()
            BiDBt = Bt.T @ iDBt

        db[tt] = {"iDBt": iDBt,
                  "zt": zt,
                  "BiDBt": BiDBt
                  }

    del iDt, Bt, iDBt, zt, BiDBt
    _ = gc.collect()
    torch.cuda.empty_cache()

    dif = float("inf")
    cnt = 0
    Z0 = data.clone()
    Z0[torch.isnan(Z0)] = 0
    old = cMLEimat(Fk=Fk, 
                   data=Z0, 
                   s=0, 
                   wSave=True,
                   device=device
                   )
    if vfixed is None:
        old["s"] = old["v"]
    else:
        old["s"] = vfixed.to(old["v"].device)
    old["M"] = convertToPositiveDefinite(mat=old["M"],
                                         device=device
                                         )
    Ptt1 = old["M"]

    while (dif > (avgtol * (100 * (ncol_Fk ** 2)))) and (cnt < maxit):
        etatt = torch.zeros((ncol_Fk, TT), device=device)
        sumPtt = torch.zeros((ncol_Fk, ncol_Fk), device=device)
        s1 = torch.zeros(TT, device=device)

        for tt in range(TT):
            iDBt = db[tt]["iDBt"].to(device)
            zt = db[tt]["zt"].to(device)
            BiDBt = db[tt]["BiDBt"].to(device)
            
            ginv_Ptt1 = torch.linalg.pinv(convertToPositiveDefinite(Ptt1))
            iP = convertToPositiveDefinite(ginv_Ptt1 + BiDBt / old["s"])
            Ptt = torch.linalg.inv(iP)  # will broken
            Gt = (Ptt @ iDBt.T) / old["s"]
            eta = Gt @ zt
            s1kk = torch.diagonal(BiDBt @ (eta.unsqueeze(1) @ eta.unsqueeze(0) + Ptt))
            
            sumPtt += Ptt
            etatt[:, tt] = eta
            s1[tt] = torch.sum(s1kk)

        if vfixed is None:
            s = torch.max(
                (torch.sum(ziDz) - 2 * torch.sum(ziDB * etatt.T) + torch.sum(s1)) / torch.sum(O),
                torch.tensor(1e-8, dtype=ziDz.dtype, device=ziDz.device)
            )
            new = {"M": (etatt @ etatt.T + sumPtt) / TT,
                   "s": s,
                   }
        else:
            new = {"M": (etatt @ etatt.T + sumPtt) / TT,
                   "s": vfixed.to(device),
                   }

        new["M"] = (new["M"] + new["M"].T) / 2
        dif = torch.sum(torch.abs(new["M"] - old["M"])) + torch.abs(new["s"] - old["s"])
        cnt += 1
        old = new
        Ptt1 = old["M"]

    if verbose:
        info_msg = f'Number of iteration: {cnt}'
        LOGGER.info(info_msg)
        
    n2loglik = computeLikelihood(data=data,
                                 Fk=Fk,
                                 M=new["M"],
                                 s=new["s"],
                                 Depsilon=Depsilon,
                                 device=device
                                 )

    if not wSave:
        return {
            "M": new["M"],
            "s": new["s"],
            "negloglik": n2loglik
        }

    elif DfromLK is not None:
        out = {
            "M": new["M"],
            "s": new["s"],
            "negloglik": n2loglik,
            "w": etatt,
            "V": new["M"] - (etatt @ etatt.T) / TT
        }

        eigenvalues, eigenvectors = torch.linalg.eigh(new["M"])
        L = Fk @ eigenvectors @ torch.diag(torch.sqrt(torch.clamp(eigenvalues, min=0.0)))

        weight = DfromLK["weights"][pick]
        wlk = torch.full((lQ.shape[0], TT), float("nan"), device=device)

        for tt in range(TT):
            obs_idx = O[:, tt].bool()
            if torch.sum(obs_idx) == O.shape[0]:
                wXiG = wwX @ torch.linalg.solve(DfromLK["G"], torch.eye(DfromLK["G"].shape[0], device=device))
            else:
                wX_tt = DfromLK["wX"][obs_idx]
                G = wX_tt.T @ wX_tt + lQ
                wXiG = wwX[obs_idx] @ torch.linalg.solve(G, torch.eye(G.shape[0], device=device))

            dat = data[obs_idx, tt]
            Lt = L[obs_idx]
            iDL = weight[obs_idx].unsqueeze(1) * Lt - wXiG @ (wwX[obs_idx].T @ Lt)
            itmp = torch.linalg.solve(
                torch.eye(L.shape[1], device=device) + (Lt.T @ iDL) / out["s"],
                torch.eye(L.shape[1], device=device)
            )
            iiLiD = itmp @ (iDL.T / out["s"])
            wlk[:, tt] = (wXiG.T @ dat - wXiG.T @ Lt @ (iiLiD @ dat)).squeeze()

        out["pinfo"] = {
            "wlk": wlk, 
            "pick": pick
        }
        out["missing"] = {
            "miss": 1 - O, 
            "maxit": maxit, 
            "avgtol": avgtol
        }
        return out

    else:
        out = {
            "M": new["M"],
            "s": new["s"],
            "negloglik": n2loglik,
            "w": etatt,
            "V": new["M"] - (etatt @ etatt.T) / TT
        }
        out["missing"] = {
            "miss": 1 - O, 
            "maxit": maxit, 
            "avgtol": avgtol
        }
        return out

# using in EM0miss
# check = ok
def isDiagonal(
    tensor: torch.Tensor,
    tol=1e-10
) -> bool:
    """
    Internal function: check if a numeric-like object is diagonal

    Parameters:
        tensor:
        tol:
    
    Return:
        bool: 
    """
    if tensor.numel() == 1:
        return True

    if tensor.ndim != 2 or tensor.shape[0] != tensor.shape[1]:
        return False

    diag = torch.diag(torch.diagonal(tensor))
    return torch.allclose(tensor, diag, atol=tol)

# using in EM0miss
# check = ok
def convertToPositiveDefinite(
    mat: torch.Tensor, 
    device: Optional[Union[torch.device, str]] = 'cpu'
) -> torch.Tensor:
    """
    Internal function: convert a matrix to positive definite

    Parameters:
        mat (torch.Tensor): Input 2D matrix (square, symmetric or not).
        device (torch.device): Device to perform computation on ("cpu" or "cuda").

    Returns:
        torch.Tensor: A positive-definite version of the input matrix.
    """
    mat = mat.to(device)

    # Ensure symmetry
    if not torch.allclose(mat, mat.T, atol=1e-10):
        mat = (mat + mat.T) / 2

    try:
        # Compute eigenvalues only
        eigenvalues = torch.linalg.eigvalsh(mat)
        min_eigenvalue = torch.min(eigenvalues).item()
    except RuntimeError:
        # Fallback in case of numerical error
        mat = (mat + mat.T) / 2
        eigenvalues = torch.linalg.eigvalsh(mat)
        min_eigenvalue = torch.min(eigenvalues).item()

    if min_eigenvalue <= 0:
        adjustment = abs(min_eigenvalue) + 1e-6
        mat = mat + torch.eye(mat.shape[0], device=device) * adjustment

    return mat

# using in EM0miss
# check = ok
def computeLikelihood(
    data: torch.Tensor,
    Fk: torch.Tensor,
    M: torch.Tensor,
    s: float,
    Depsilon: torch.Tensor,
    device: Optional[Union[torch.device, str]] = 'cpu'
) -> float:
    """
    Compute negative log-likelihood (-2 * log(likelihood)).

    Parameters:
        data (n x T): Observation matrix with possible NaNs.
        Fk (n x K): Basis function matrix.
        M (K x K): Symmetric matrix.
        s (float): Scalar multiplier.
        Depsilon (n x n): Diagonal matrix.
        device: CPU or GPU.

    Returns:
        float: Negative log-likelihood value.
    """
    data = data.to(device)
    Fk = Fk.to(device)
    M = M.to(device)
    Depsilon = Depsilon.to(device)

    non_missing_points_matrix = ~torch.isnan(data)
    num_columns = data.shape[1]

    n2loglik = non_missing_points_matrix.sum() * torch.log(torch.tensor(2 * torch.pi, device=device))
    R = s * Depsilon
    eg = eigenDecompose(M,
                        device=device
                        )
    K = Fk.shape[1]
    L = Fk @ eg["vector"] @ torch.diag(torch.sqrt(torch.clamp(eg["value"], min=0.0))) @ eg["vector"].T
    
    for t in range(num_columns):
        mask = non_missing_points_matrix[:, t]
        zt = data[mask, t]

        # skip all-missing column
        if zt.numel() == 0:
            continue

        Rt = R[mask][:, mask]
        Lt = L[mask]

        log_det = calculateLogDeterminant(Rt, 
                                          Lt, 
                                          K, 
                                          device=device
                                          )
        inv_cz_val = invCz(Rt, 
                           Lt, 
                           zt, 
                           device=device
                           )
        n2loglik += log_det + torch.sum(zt * inv_cz_val)

    return n2loglik.item()

# using in computeLikelihood
# check = ok
def eigenDecompose(
    matrix: torch.Tensor,
    device: Optional[Union[torch.device, str]] = 'cpu'
) -> dict:
    """
    Internal function: Eigen-decompose a matrix

    Parameters:
        matrix (torch.Tensor): (K x K) symmetric matrix
        device (str or torch.device): computation device

    Returns:
        dict with keys:
            'value': (K,) tensor of eigenvalues
            'vector': (K x K) tensor of eigenvectors (columns)
    """
    matrix = matrix.to(device)

    # Use symmetric eigendecomposition
    eigenvalues, eigenvectors = torch.linalg.eigh(matrix)

    return {
        'value': eigenvalues,
        'vector': eigenvectors
    }

# using in computeLikelihood
# check = ok
def calculateLogDeterminant(
    R: torch.Tensor,
    L: torch.Tensor,
    K: int=None,
    device: Optional[Union[str, torch.device]] = 'cpu'
) -> float:
    """
    Internal function: calculate the log determinant for the likelihood use.

    Parameters:
        R (torch.Tensor): (p x p) positive-definite matrix
        L (torch.Tensor): (p x K) matrix
        K (int): A numeric
        device (str or torch.device): computation device

    Returns:
        float: log-determinant value
    """
    R = R.to(device)
    L = L.to(device)

    if K is None:
        K = L.shape[1]

    first_part_determinant = torch.logdet(torch.eye(K, device=device) + L.T @ torch.linalg.solve(R, L))
    second_part_determinant = torch.logdet(R)

    return (first_part_determinant + second_part_determinant).item()

# using in indeMLE


In [27]:
def cMLE(
    Fk: torch.Tensor,
    num_columns: int,
    sample_covariance_trace: float,
    inverse_square_root_matrix: torch.Tensor,
    matrix_JSJ: torch.Tensor,
    s: float = 0,
    ldet: float = 0,
    wSave: bool = False,
    onlylogLike: bool = None,
    vfixed: float = None,
    device: Optional[Union[torch.device, str]] = 'cpu'
) -> dict:
    """
    Internal function: maximum likelihood estimate with the likelihood

    Parameters:
        Fk: (N, K) torch.Tensor, basis functions.
        num_columns: (int) Number of columns in the data.
        sample_covariance_trace: (float) Trace of the sample covariance matrix.
        inverse_square_root_matrix: (N, K) torch.Tensor, inverse square root matrix.
        matrix_JSJ: (K, K) torch.Tensor, covariance-like matrix.
        s: (float) Effective sample size, default is 0.
        ldet: (float) Log determinant of the transformation matrix, default is 0.
        wSave: (bool) Whether to save the L matrix, default is False.
        onlylogLike: (bool) If True, only return the negative log likelihood.
        vfixed: (float, optional) Fixed noise variance, if provided.
        device: (str) 'cpu' or 'cuda'.

    Returns:
        dict: {
            'v': (float) Estimated noise variance,
            'M': (torch.Tensor) Matrix M,
            's': (int) Effective sample size,
            'negloglik': (float) Negative log likelihood,
            'L': (torch.Tensor, optional) L matrix if wSave is True.
        }
    """
    nrow_Fk = Fk.shape[0]

    likelihood_object = computeNegativeLikelihood(
        nrow_Fk=nrow_Fk,
        ncol_Fk=Fk.shape[1],
        s=s,
        p=num_columns,
        matrix_JSJ=matrix_JSJ,
        sample_covariance_trace=sample_covariance_trace,
        vfixed=vfixed,
        ldet=ldet,
        device=device
    )

    negative_log_likelihood = likelihood_object['negative_log_likelihood']

    if onlylogLike:
        return {'negloglik': negative_log_likelihood}

    P = likelihood_object['P']
    d_hat = likelihood_object['d_hat']
    v = likelihood_object['v']
    M = inverse_square_root_matrix @ P @ (torch.diag(d_hat) @ P.T) @ inverse_square_root_matrix

    if not wSave:
        L = None
    elif d_hat[0] != 0:
        L = Fk @ ((torch.diag(torch.sqrt(d_hat)) @ P.T) @ inverse_square_root_matrix)
        L = L[:, d_hat > 0]
    else:
        L = torch.zeros((nrow_Fk, 1), dtype=Fk.dtype, device=Fk.device)

    return {'v': v,
            'M': M,
            's': s,
            'negloglik': negative_log_likelihood,
            'L': L
            }

In [21]:
# ==== 固定測試資料 ====
import torch, numpy as np
device = "cpu"; dtype = torch.float64

Fk = torch.tensor([
    [0.1, 0.2, 0.3],
    [0.4, 0.5, 0.6],
    [0.7, 0.8, 0.9],
    [1.0, 1.1, 1.2],
    [1.3, 1.4, 1.5]
], dtype=dtype, device=device)

Z0 = torch.tensor([
    [1, 0, 2],
    [2, 3, 0],
    [0, 1, 4],
    [3, 2, 1],
    [4, 3, 0]
], dtype=dtype, device=device)

# ==== 列印工具（逐元素，以 "name[i]" / "name[i,j]" = value 格式）====
def print_scalar(name, x, precision=17):
    if isinstance(x, torch.Tensor): x = x.item()
    print(f"{name} = {x:.{precision}g}")

def print_vector(name, t, precision=17):
    a = t.detach().cpu().numpy() if isinstance(t, torch.Tensor) else np.asarray(t)
    a = a.reshape(-1)
    for i, v in enumerate(a):
        print(f"{name}[{i}] = {v:.{precision}g}")

def print_matrix(name, t, precision=17):
    if t is None:
        print(f"{name} = NULL"); return
    A = t.detach().cpu().numpy() if isinstance(t, torch.Tensor) else np.asarray(t)
    R, C = A.shape
    for i in range(R):
        for j in range(C):
            print(f"{name}[{i},{j}] = {A[i,j]:.{precision}g}")

# ==== cMLE 相關步驟 ====
proj = computeProjectionMatrix(Fk, Fk, Z0, device=device)
sample_cov_trace = float(torch.sum(Z0**2).item() / Z0.shape[1])

# 先拿 likelihood 物件（為了取 P、d_hat、v、negloglik）
lik = computeNegativeLikelihood(
    nrow_Fk=Fk.shape[0],
    ncol_Fk=Fk.shape[1],
    s=0, p=Z0.shape[1],
    matrix_JSJ=proj['matrix_JSJ'],
    sample_covariance_trace=sample_cov_trace,
    vfixed=None, ldet=0.0, device=device
)

# 用 cMLE 得到 M、L（就是你要檢查的重點）
out = cMLE(
    Fk=Fk,
    num_columns=Z0.shape[1],
    sample_covariance_trace=sample_cov_trace,
    inverse_square_root_matrix=proj['inverse_square_root_matrix'],
    matrix_JSJ=proj['matrix_JSJ'],
    s=0, ldet=0.0, wSave=True, onlylogLike=False, device=device
)

# ==== 逐行列印：完整覆蓋 R 版左側與 return 的元素 ====
print_scalar("nrow_Fk", Fk.shape[0])

print_scalar("negative_log_likelihood", lik['negative_log_likelihood'])
print_matrix("P", lik['P'])
print_vector("d_hat", lik['d_hat'])
print_scalar("v", lik['v'])

print_matrix("M", out['M'])
print_matrix("L", out['L'])   # 若 None 會印成 NULL
print_scalar("s", 0)
print_scalar("negloglik", lik['negative_log_likelihood'])


nrow_Fk = 5
negative_log_likelihood = 59.230576791600257
P[0,0] = -0.37883572409221677
P[0,1] = -0.83055212202396778
P[0,2] = 0.40824829056961093
P[1,0] = -0.5584200704187291
P[1,1] = -0.14663000485615038
P[1,2] = -0.81649658090492128
P[2,0] = -0.73800441672463435
P[2,1] = 0.537292112614143
P[2,2] = 0.40824829040372412
d_hat[0] = 14.756686031642779
d_hat[1] = 0
d_hat[2] = 0
v = 1.9819961270047779
M[0,0] = 4100994763.8106313
M[0,1] = -8199995663.3397942
M[0,2] = 4099235889.5982118
M[1,0] = -8199995663.3397932
M[1,1] = 16396004567.51432
M[1,2] = -8196478770.0627089
M[2,0] = 4099235889.5982113
M[2,1] = -8196478770.0627098
M[2,2] = 4097477769.7487383
L[0,0] = -5220.4286690646959
L[1,0] = -20611.683800526738
L[2,0] = -36002.938931988778
L[3,0] = -51394.194063450828
L[4,0] = -66785.449194912857
s = 0
negloglik = 59.230576791600257


In [28]:
import torch, numpy as np
dtype = torch.float64; device = "cpu"

# ---- R 那邊印出來的 J、JSJ（照你貼的數值）----
J_R = torch.tensor([
    [ 5998983.7193301283 , -11997959.333595194,  5998976.5695005218 ],
    [-11997959.333595185 ,  23995919.412441969, -11997959.753569832 ],
    [ 5998976.5695005143 , -11997959.753569825,  5998982.8793871226 ],
], dtype=dtype, device=device)

JSJ_R = torch.tensor([
    [3.3413369999902756 , 3.7068460514642916 , 4.0723548485718872 ],
    [3.7068460514642916 , 5.2489414732442201 , 6.791037462524776  ],
    [4.0723548485718872 , 6.791037462524776  , 9.5097214658459919 ],
], dtype=dtype, device=device)

# ---- 與 R 相同的 Fk、Z0 ----
Fk = torch.tensor([
    [0.1, 0.2, 0.3],
    [0.4, 0.5, 0.6],
    [0.7, 0.8, 0.9],
    [1.0, 1.1, 1.2],
    [1.3, 1.4, 1.5]
], dtype=dtype, device=device)

Z0 = torch.tensor([
    [1, 0, 2],
    [2, 3, 0],
    [0, 1, 4],
    [3, 2, 1],
    [4, 3, 0]
], dtype=dtype, device=device)

# ---- 列印工具（0-based）----
def print_scalar(name, x, precision=17):
    if isinstance(x, torch.Tensor): x = x.item()
    print(f"{name} = {x:.{precision}g}")
def print_vector(name, t, precision=17):
    a = t.detach().cpu().numpy().reshape(-1)
    for i, v in enumerate(a):
        print(f"{name}[{i}] = {v:.{precision}g}")
def print_matrix(name, t, precision=17):
    A = t.detach().cpu().numpy()
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            print(f"{name}[{i},{j}] = {A[i,j]:.{precision}g}")

# ---- 跟 R 相同的 sample_cov_trace ----
sample_cov_trace = float(torch.sum(Z0**2) / Z0.shape[1])

# ---- 用你現有的 computeNegativeLikelihood（不改）----
lik = computeNegativeLikelihood(
    nrow_Fk=Fk.shape[0],
    ncol_Fk=Fk.shape[1],
    s=0, p=Z0.shape[1],
    matrix_JSJ=JSJ_R,
    sample_covariance_trace=sample_cov_trace,
    vfixed=None, ldet=0.0, device=device
)
print_matrix("P_py", lik['P'])
print_vector("d_hat_py", lik['d_hat'])
print_scalar("v_py", lik['v'])
print_scalar("negative_log_likelihood_py", lik['negative_log_likelihood'])

# ---- 用「原始、不改動」的 cMLE 跑（只換輸入成 R 的 J/JSJ）----
out_raw = cMLE(
    Fk=Fk,
    num_columns=Z0.shape[1],
    sample_covariance_trace=sample_cov_trace,
    inverse_square_root_matrix=J_R,
    matrix_JSJ=JSJ_R,
    s=0, ldet=0.0, wSave=True, onlylogLike=False, device=device
)
print_matrix("M_from_cMLE_raw", out_raw['M'])
print_matrix("L_from_cMLE_raw", out_raw['L'])
print_scalar("s_py", out_raw['s'])
print_scalar("negloglik_py", out_raw['negloglik'])
print_scalar("v_py_out", out_raw['v'])


P_py[0,0] = -0.37883573366421197
P_py[0,1] = -0.8305520162969896
P_py[0,2] = 0.40824849678119213
P_py[1,0] = -0.55842004510022214
P_py[1,1] = -0.14663022080531096
P_py[1,2] = -0.81649655943969024
P_py[2,0] = -0.7380044309686522
P_py[2,1] = 0.53729221711429098
P_py[2,2] = 0.40824812712252001
d_hat_py[0] = 14.756685964424952
d_hat_py[1] = 0
d_hat_py[2] = 0
v_py = 1.9819961404483433
negative_log_likelihood_py = 59.230576863356646
M_from_cMLE_raw[0,0] = 1.3642586799576226
M_from_cMLE_raw[0,1] = 3.5303392486700833
M_from_cMLE_raw[0,2] = -6.3243891792387847
M_from_cMLE_raw[1,0] = 3.5303392531547355
M_from_cMLE_raw[1,1] = 9.1355806492093503
M_from_cMLE_raw[1,2] = -16.36584005826959
M_from_cMLE_raw[2,0] = -6.3243891768574061
M_from_cMLE_raw[2,1] = -16.365840028932933
M_from_cMLE_raw[2,2] = 29.318412350417312
L_from_cMLE_raw[0,0] = 0.11680148462319535
L_from_cMLE_raw[1,0] = 0.46720593849278141
L_from_cMLE_raw[2,0] = 0.81761039236236743
L_from_cMLE_raw[3,0] = 1.1680148462319535
L_from_cMLE_raw[4

In [26]:
# ========= 對照驗證：函式外用 diag 寫法重算 M / L =========
import torch
import numpy as np

# 列印工具（0-based）
def print_matrix(name, t, precision=17):
    A = t.detach().cpu().numpy()
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            print(f"{name}[{i},{j}] = {A[i,j]:.{precision}g}")

# 1) 取得需要的變數：
# - P、d_hat 來自你剛跑完的 likelihood 物件（lik）
# - J 請用你「想要對齊 R」的那個 inverse_square_root_matrix：
#   若你已把 R 的 J 放在 J_R，就用 J_R；否則就用 proj['inverse_square_root_matrix']。
P = lik['P']
d_hat = lik['d_hat']

# 選一個（優先用 R 的 J 做對照）
J = J_R if 'J_R' in globals() else proj['inverse_square_root_matrix']

# 2) 用 R 的等價公式（列縮放）重算 M / L：
M_diag = J @ P @ (torch.diag(d_hat) @ P.T) @ J

L_diag_full = Fk @ (torch.diag(torch.sqrt(d_hat)) @ P.T @ J)
L_diag = L_diag_full[:, d_hat > 0]   # 與 R 一樣只保留 d_hat > 0 的欄

# 3) 列印結果（拿去和 R 的 M / L 對照）
print_matrix("M_diag_check", M_diag)
print_matrix("L_diag_check", L_diag)


M_diag_check[0,0] = 1.3642586799576226
M_diag_check[0,1] = 3.5303392486700833
M_diag_check[0,2] = -6.3243891792387847
M_diag_check[1,0] = 3.5303392531547355
M_diag_check[1,1] = 9.1355806492093503
M_diag_check[1,2] = -16.36584005826959
M_diag_check[2,0] = -6.3243891768574061
M_diag_check[2,1] = -16.365840028932933
M_diag_check[2,2] = 29.318412350417312
L_diag_check[0,0] = 0.11680148462319535
L_diag_check[1,0] = 0.46720593849278141
L_diag_check[2,0] = 0.81761039236236743
L_diag_check[3,0] = 1.1680148462319535
L_diag_check[4,0] = 1.5184193001015396
